In [303]:
import pandas as pd

In [304]:
inventory = pd.read_csv("../inventory/inventory_state.csv")
customers = pd.read_csv("../data/processed/customer_segments.csv")
transactions = pd.read_csv("../data/processed/cleaned.csv")

In [305]:
transactions = transactions.merge(customers[['Customer ID','ClusterLabel']], on="Customer ID")

In [306]:
transactions

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,ClusterLabel
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,Regular Customers
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Regular Customers
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Regular Customers
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,Regular Customers
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,Regular Customers
...,...,...,...,...,...,...,...,...,...
776681,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,New Customers
776682,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,New Customers
776683,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,New Customers
776684,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,New Customers


In [307]:
product_cluster = (
    transactions.groupby(["StockCode", "ClusterLabel"])
    .size()
    .reset_index(name="count")
)

total_per_product = (
    product_cluster.groupby("StockCode")["count"]
    .sum()
    .reset_index(name="total")
)

product_cluster = product_cluster.merge(total_per_product, on="StockCode")
product_cluster["share"] = product_cluster["count"] / product_cluster["total"]


In [308]:
def choose_target_cluster(df_product):
    # df_product is all rows for one StockCode
    
    cluster_share = dict(zip(df_product["ClusterLabel"], df_product["share"]))

    at_risk = cluster_share.get("At-Risk Customers", 0)
    new = cluster_share.get("New Customers", 0)
    vip = cluster_share.get("VIP Customers", 0)
    regular = cluster_share.get("Regular Customers", 0)

    # Marketing priority logic
    if at_risk >= 0.15:
        return "At-Risk Customers"
    elif new >= 0.20:
        return "New Customers"
    elif vip >= 0.10:
        return "VIP Customers"
    else:
        return "Regular Customers"


In [309]:
target_clusters = (
    product_cluster
    .groupby("StockCode")
    .apply(choose_target_cluster)
    .reset_index()
)

target_clusters.columns = ["StockCode", "TargetCluster"]


C:\Users\bsais\AppData\Local\Temp\ipykernel_15232\975722114.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(choose_target_cluster)


In [310]:
df = inventory.merge(target_clusters, on="StockCode", how="left")

df["OverstockRatio"] = df["Current_Stock"] / df["Predicted_7d_Demand"]

In [311]:
df = inventory.merge(target_clusters, on="StockCode", how="left")

In [312]:
df = df[df["Predicted_7d_Demand"] > 0]

In [313]:
DEMAND_THRESHOLD = 30

In [314]:
df["DemandLevel"] = df["Predicted_7d_Demand"].apply(
    lambda x: "High" if x >= DEMAND_THRESHOLD else "Low"
)


In [315]:
df["OverstockRatio"] = df["Current_Stock"] / df["Predicted_7d_Demand"]
df["IsOverstock"] = df["OverstockRatio"] > 1.5

In [316]:
def get_objective(row):
    if row["IsOverstock"] and row["DemandLevel"] == "Low":
        return "Clear Stock"
    elif row["IsOverstock"] and row["DemandLevel"] == "High":
        return "Customer Acquisition"
    elif not row["IsOverstock"] and row["DemandLevel"] == "High":
        return "Maximize Profit"
    else:
        return "Retention"

df["Objective"] = df.apply(get_objective, axis=1)

In [317]:
def choose_cluster(row):
    if row["Objective"] == "Clear Stock":
        return "At-Risk Customers"
    elif row["Objective"] == "Customer Acquisition":
        return "New Customers"
    elif row["Objective"] == "Maximize Profit":
        return "VIP Customers"
    else:
        return "Regular Customers"

In [318]:
df["TargetCluster"] = df.apply(choose_cluster, axis=1)

In [319]:
def get_discount(row):
    r = row["OverstockRatio"]
    obj = row["Objective"]

    if obj == "Clear Stock":
        if r > 2.5:
            return 60
        elif r > 2.0:
            return 50
        else:
            return 40

    elif obj == "Customer Acquisition":
        if r > 2.0:
            return 30
        else:
            return 20

    elif obj == "Maximize Profit":
        return 5   # VIP exclusivity, no margin killing

    elif obj == "Retention":
        return 10

In [320]:
df["Discount"] = df.apply(get_discount, axis=1)


In [321]:
MIN_DEMAND = 10
MIN_STOCK = 15

In [322]:
df = df[
    (df["Predicted_7d_Demand"] >= MIN_DEMAND) &
    (df["Current_Stock"] >= MIN_STOCK)
]

In [323]:
df = df[
    df["Objective"].isin(["Clear Stock", "Customer Acquisition", "Maximize Profit"])
]

In [324]:
final_campaigns = df[[
    "StockCode",
    "Predicted_7d_Demand",
    "Current_Stock",
    "OverstockRatio",
    "Objective",
    "TargetCluster",
    "Discount"
]]

In [325]:
regular_engine = inventory.copy()
regular_engine = regular_engine[regular_engine["Predicted_7d_Demand"] > 30]

In [326]:
regular_engine["OverstockRatio"] = (
    regular_engine["Current_Stock"] / regular_engine["Predicted_7d_Demand"]
)


In [327]:
regular_engine = regular_engine[
    (regular_engine["OverstockRatio"] >= 0.8) &
    (regular_engine["OverstockRatio"] <= 1.5)
]

In [328]:
regular_engine["Objective"] = "Stability Revenue"
regular_engine["TargetCluster"] = "Regular Customers"
regular_engine["Discount"] = 5   # loyalty style


In [329]:
regular_campaigns = regular_engine[[
    "StockCode",
    "Predicted_7d_Demand",
    "Current_Stock",
    "OverstockRatio",
    "Objective",
    "TargetCluster",
    "Discount"
]]

In [330]:
all_campaigns = pd.concat(
    [final_campaigns, regular_campaigns],
    ignore_index=True
)

In [331]:
lost_campaigns = all_campaigns[
    (all_campaigns["TargetCluster"] == "At-Risk Customers") &
    (all_campaigns["Discount"] >= 50)
].copy()

lost_campaigns["TargetCluster"] = "Lost Customers"
lost_campaigns["Objective"] = "Win Back & Clear Stock"

In [332]:
all_campaigns = pd.concat(
    [all_campaigns, lost_campaigns],
    ignore_index=True
)

In [344]:
all_campaigns.shape

(939, 7)

In [334]:
all_campaigns[all_campaigns['TargetCluster'] == 'Lost Customers']

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
815,10135,10.892109,26,2.387049,Win Back & Clear Stock,Lost Customers,50
816,16014,14.270042,34,2.382614,Win Back & Clear Stock,Lost Customers,50
817,16218,17.107687,38,2.221224,Win Back & Clear Stock,Lost Customers,50
818,20749,18.847027,42,2.228468,Win Back & Clear Stock,Lost Customers,50
819,20956,10.328640,23,2.226818,Win Back & Clear Stock,Lost Customers,50
...,...,...,...,...,...,...,...
934,84535B,11.064857,24,2.169029,Win Back & Clear Stock,Lost Customers,50
935,84971S,11.711815,24,2.049213,Win Back & Clear Stock,Lost Customers,50
936,85034B,11.311345,25,2.210170,Win Back & Clear Stock,Lost Customers,50
937,85049G,15.641891,35,2.237581,Win Back & Clear Stock,Lost Customers,50


In [335]:
all_campaigns[all_campaigns['TargetCluster'] == 'VIP Customers']

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
2,15036,78.675980,87,1.105801,Maximize Profit,VIP Customers,5
3,15056BL,30.694668,33,1.075105,Maximize Profit,VIP Customers,5
4,15056N,35.394714,43,1.214871,Maximize Profit,VIP Customers,5
7,16045,51.429504,76,1.477751,Maximize Profit,VIP Customers,5
9,16161P,53.623660,68,1.268097,Maximize Profit,VIP Customers,5
...,...,...,...,...,...,...,...
660,85048,30.032013,18,0.599360,Maximize Profit,VIP Customers,5
661,85049A,60.610325,34,0.560961,Maximize Profit,VIP Customers,5
669,85099F,131.102690,142,1.083120,Maximize Profit,VIP Customers,5
671,85150,39.635483,46,1.160576,Maximize Profit,VIP Customers,5


In [336]:
all_campaigns[all_campaigns['TargetCluster'] == 'Regular Customers']

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
675,15036,78.675980,87,1.105801,Stability Revenue,Regular Customers,5
676,15056BL,30.694668,33,1.075105,Stability Revenue,Regular Customers,5
677,15056N,35.394714,43,1.214871,Stability Revenue,Regular Customers,5
678,16045,51.429504,76,1.477751,Stability Revenue,Regular Customers,5
679,16161P,53.623660,68,1.268097,Stability Revenue,Regular Customers,5
...,...,...,...,...,...,...,...
810,84992,102.044500,136,1.332752,Stability Revenue,Regular Customers,5
811,84997B,31.449594,40,1.271877,Stability Revenue,Regular Customers,5
812,85099F,131.102690,142,1.083120,Stability Revenue,Regular Customers,5
813,85150,39.635483,46,1.160576,Stability Revenue,Regular Customers,5


In [337]:
all_campaigns[all_campaigns['TargetCluster'] == 'At-Risk Customers']

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
0,10133,10.275503,19,1.849058,Clear Stock,At-Risk Customers,40
1,10135,10.892109,26,2.387049,Clear Stock,At-Risk Customers,50
5,16008,18.866154,32,1.696159,Clear Stock,At-Risk Customers,40
6,16014,14.270042,34,2.382614,Clear Stock,At-Risk Customers,50
11,16216,10.007176,17,1.698781,Clear Stock,At-Risk Customers,40
...,...,...,...,...,...,...,...
662,85049C,11.218756,21,1.871865,Clear Stock,At-Risk Customers,40
664,85049G,15.641891,35,2.237581,Clear Stock,At-Risk Customers,50
665,85061W,16.638247,30,1.803075,Clear Stock,At-Risk Customers,40
666,85071B,12.975546,32,2.466178,Clear Stock,At-Risk Customers,50


In [338]:
all_campaigns[all_campaigns['TargetCluster'] == 'New Customers']

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
8,16156S,42.364445,99,2.336865,Customer Acquisition,New Customers,30
10,16169E,77.300810,191,2.470867,Customer Acquisition,New Customers,30
14,16237,37.303234,60,1.608440,Customer Acquisition,New Customers,20
17,20676,33.457590,53,1.584095,Customer Acquisition,New Customers,20
19,20685,37.188137,81,2.178114,Customer Acquisition,New Customers,30
...,...,...,...,...,...,...,...
663,85049E,48.289265,86,1.780934,Customer Acquisition,New Customers,20
667,85099B,627.295040,1430,2.279629,Customer Acquisition,New Customers,30
668,85099C,149.639200,371,2.479297,Customer Acquisition,New Customers,30
670,85123A,504.970980,951,1.883277,Customer Acquisition,New Customers,20


In [339]:
all_campaigns[all_campaigns['Discount'] == 50]

,StockCode,Predicted_7d_Demand,Current_Stock,OverstockRatio,Objective,TargetCluster,Discount
1,10135,10.892109,26,2.387049,Clear Stock,At-Risk Customers,50
6,16014,14.270042,34,2.382614,Clear Stock,At-Risk Customers,50
12,16218,17.107687,38,2.221224,Clear Stock,At-Risk Customers,50
31,20749,18.847027,42,2.228468,Clear Stock,At-Risk Customers,50
36,20956,10.328640,23,2.226818,Clear Stock,At-Risk Customers,50
...,...,...,...,...,...,...,...
934,84535B,11.064857,24,2.169029,Win Back & Clear Stock,Lost Customers,50
935,84971S,11.711815,24,2.049213,Win Back & Clear Stock,Lost Customers,50
936,85034B,11.311345,25,2.210170,Win Back & Clear Stock,Lost Customers,50
937,85049G,15.641891,35,2.237581,Win Back & Clear Stock,Lost Customers,50


In [ ]:
all_campaigns.to_csv('../data/processed/campaign_plan.csv', index=False)